In [1]:
from pathlib import Path
import json
import os

from tinydb import TinyDB

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)

class KVDB(object):
    def __init__(self, db_path):
        self._db_path = Path(db_path)
        self._db = {}
        self._load_db()

    def _load_db(self):
        if self._db_path.exists():
            with open(self._db_path) as f:
                self._db = json.load(f)

    def get_value(self, key):
        return self._db.get(key)

    def set_value(self, key, value):
        self._db[key] = value

    def save(self):
        with open(self._db_path, 'w') as f:
            json.dump(self._db, f, indent=2)
            
    def get_table(self):
        return self._db
    


class DocumentDB(object):
    def __init__(self, db_path):
        ## You can use the code from the previous exmaple if you would like
        people_json = kv_data_dir.joinpath('people.json')
        visited_json = kv_data_dir.joinpath('visited.json')
        sites_json = kv_data_dir.joinpath('sites.json')
        measurements_json = kv_data_dir.joinpath('measurements.json')

        self._db_path = Path(db_path)
        self._db = None
        
        ## TODO: Implement code
        self.people = KVDB(people_json).get_table()       
        self.visited = KVDB(visited_json).get_table()
        self.sites = KVDB(sites_json).get_table()
        self.measurements = KVDB(measurements_json).get_table()
        
        self.patient_info_list = []
        self.visited_update ={}
        
        for k,v in self.visited.items():
            new_key = DocumentDB._split_composite_key(k)
            self.visited_update[int(new_key[0])] = v
            
        
        for k,v in self.people.items():
            patient_individual_dict = {}
            for p_k, p_v in v.items(): 
                patient_individual_dict[p_k] = p_v
            patient_individual_dict['visits'] = self._lookup_visits(k)
            indent_values = json.dumps(patient_individual_dict, indent = 2)
            self.patient_info_list.append(patient_individual_dict)
            
        self._load_db()
        
    def _lookup_site(self, site_id):
        site_dict = {}
        for k,v in self.sites[site_id].items():
            site_dict[k] = v
        return site_dict
            
        
    def _lookup_visits(self, person_id):
        measures = self._lookup_measures(person_id)
        visit_list = []
        for k,v in measures.items():
            visit_dict = {}
            site_dict = {}
            for v_k,v_v in self.visited_update[k].items():
                visit_dict[v_k] = v_v
                if v_k == 'site_id': 
                    site_dict = self._lookup_site(v_v)
            visit_dict['site'] = site_dict
            visit_dict['measurements'] = v
            if len(measures) > 1:
                visit_list.append(visit_dict)
            else: 
                return visit_list 
        return visit_list    
    
    def _lookup_measures(self, person_id):
        list_of_measures = {}
        for k,v in self.measurements.items():
            if v['person_id'] == person_id:
                try:
                    if isinstance(list_of_measures[v['visit_id']], dict):
                        list_of_measures[v['visit_id']] = [list_of_measures[v['visit_id']]]
                        list_of_measures[v['visit_id']].append(v) 
                    else:
                        list_of_measures[v['visit_id']].append(v)                       
                except:
                    list_of_measures[v['visit_id']] = v            
        return list_of_measures

    def _load_db(self):
        self._db = TinyDB(self._db_path)
        self._db.insert_multiple(self.patient_info_list)

    #staticmethod
    def _split_composite_key(key):
        return key.replace('(','').replace(')', '').replace(' ', '').split(',')
    
    
        
              
DocumentDB('results\patient-info.json')